In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

In [2]:
df = pd.read_json('transactions.txt', lines=True)

df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,,0.0,,,,False,,,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,,0.0,,,,True,,,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,,0.0,,,,True,,,False,False


## Data Cleaning

In [3]:
# Replacing white space with nan

df = df.replace('', np.nan)

In [4]:
# Checking which columns are empty and removing

nan_col = df.columns[df.isna().all()].tolist()
df.drop(columns=nan_col, inplace=True)

print("Empty columns that were removed:")
print(nan_col)

Empty columns that were removed:
['echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd']


In [5]:
# Account Number and Customer Id columns appear to be equal

df['accountNumber'].equals(df['customerId'])

True

In [6]:
# Removing customerId as it is a duplicate of accountNumber

df.drop(columns=['customerId'], inplace=True)

In [7]:
# Dropping merchantName as merchantCategory will be used
df.drop(columns=['merchantName'], inplace=True)

In [8]:
# Need to remove remaining unneeded columns
#df.drop(columns=['cardCVV'], inplace=True)
#df.drop(columns=['enteredCVV'], inplace=True)
df.drop(columns=['cardLast4Digits'], inplace=True)

In [9]:
# Filling NaN with purchase to keep data

df['transactionType'] = df['transactionType'].fillna('PURCHASE')

In [10]:
# Total amount of rows with NaN values
df.isnull().any(axis=1).sum()

9003

In [11]:
# Dropping all rows with NaN value
df = df.dropna()

In [12]:
pd.pandas.set_option('display.max_columns', None)

In [13]:
# Changing transactionDatetime. All transactions occurred in 2016, year is unneeded.

df['transactionDateTime'] = pd.to_datetime(df['transactionDateTime'])
df['Month'] = df['transactionDateTime'].dt.month
df['day'] = df['transactionDateTime'].dt.day
df['hour'] = df['transactionDateTime'].dt.hour
df['minute'] = df['transactionDateTime'].dt.minute

# Cyclic encoding for hour and minute
df['hourSin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hourCos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minuteSin'] = np.sin(2 * np.pi * df['minute'] / 24)
df['minuteCos'] = np.cos(2 * np.pi * df['minute'] / 24)

# Do not need columns
df.drop(columns=['transactionDateTime'], inplace=True)
df.drop(columns=['hour'], inplace=True)
df.drop(columns=['minute'], inplace=True)

In [14]:
# Reformatting other date columns

df['expYear'] = pd.to_datetime(df['currentExpDate']).dt.year
df['expMonth'] = pd.to_datetime(df['currentExpDate']).dt.month

df['accountOpenYear'] = pd.to_datetime(df['accountOpenDate']).dt.year
df['accountOpenMonth'] = pd.to_datetime(df['accountOpenDate']).dt.month
df['accountOpenDay'] = pd.to_datetime(df['accountOpenDate']).dt.day

df['lastAddressChangeYear'] = pd.to_datetime(df['dateOfLastAddressChange']).dt.year
df['lastAddressChangeMonth'] = pd.to_datetime(df['dateOfLastAddressChange']).dt.month
df['lastAddressChangeDay'] = pd.to_datetime(df['dateOfLastAddressChange']).dt.day

# removing unneeded columns
df.drop(columns=['currentExpDate'], inplace=True)
df.drop(columns=['accountOpenDate'], inplace=True)
df.drop(columns=['dateOfLastAddressChange'], inplace=True)

## Encoding

In [15]:
# Categorical columns that need to be endocded

columns_to_encode = ['acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode', 'transactionType', 'merchantCategoryCode']
df = pd.get_dummies(df, columns=columns_to_encode, prefix=columns_to_encode)

# Convert boolean columns to integers
df['cardPresent'] = df['cardPresent'].astype(int)
df['expirationDateKeyInMatch'] = df['expirationDateKeyInMatch'].astype(int)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777360 entries, 0 to 786362
Data columns (total 62 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   accountNumber                              777360 non-null  int64  
 1   creditLimit                                777360 non-null  int64  
 2   availableMoney                             777360 non-null  float64
 3   transactionAmount                          777360 non-null  float64
 4   cardCVV                                    777360 non-null  int64  
 5   enteredCVV                                 777360 non-null  int64  
 6   currentBalance                             777360 non-null  float64
 7   cardPresent                                777360 non-null  int32  
 8   expirationDateKeyInMatch                   777360 non-null  int32  
 9   isFraud                                    777360 non-null  bool   
 10  Month   

In [17]:
# Breaking apart features and labels

features = df.iloc[:, :-1]
labels = df['isFraud']

## Splitting dataset into train, test and val

In [18]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X = df.drop('isFraud', axis = 1)
Y = df['isFraud']

x_train, x_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.7, random_state=333, stratify=Y)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=333, stratify=y_temp)


In [19]:
#standardize/normalize your numerical features
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

#use DataFrame.select_dtypes() to select float64 or int64 feature types automatically.
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns # Get a list of the column names
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

#Fit your instance ct of ColumnTransformer to the training data and at the same time transform it by using the ColumnTransformer.fit_transform()
x_train_scaled = ct.fit_transform(x_train)
x_test_scaled = ct.fit_transform(x_test) #Transform your test data instance features_test
x_val_scaled = ct.fit_transform(x_val)

## Model Creation

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [21]:
model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall()]
)

In [22]:
history = model.fit(x_train_scaled, y_train, 
                    validation_data=(x_val_scaled, y_val),
                    epochs=10,
                    batch_size=32)

Epoch 1/10

7288/7288 [==============================] - 13s 2ms/step - loss: 0.0754 - binary_accuracy: 0.9844 - recall: 2.7824e-04 - val_loss: 0.0740 - val_binary_accuracy: 0.9846 - val_recall: 0.0000e+00
Epoch 2/10
7288/7288 [==============================] - 11s 2ms/step - loss: 0.0726 - binary_accuracy: 0.9846 - recall: 0.0000e+00 - val_loss: 0.0738 - val_binary_accuracy: 0.9846 - val_recall: 0.0000e+00
Epoch 3/10
7288/7288 [==============================] - 11s 2ms/step - loss: 0.0715 - binary_accuracy: 0.9846 - recall: 0.0000e+00 - val_loss: 0.0718 - val_binary_accuracy: 0.9846 - val_recall: 0.0000e+00
Epoch 4/10
7288/7288 [==============================] - 12s 2ms/step - loss: 0.0707 - binary_accuracy: 0.9846 - recall: 0.0000e+00 - val_loss: 0.0727 - val_binary_accuracy: 0.9846 - val_recall: 0.0000e+00
Epoch 5/10
7288/7288 [==============================] - 12s 2ms/step - loss: 0.0699 - binary_accuracy: 0.9846 - recall: 0.0000e+00 - val_loss: 0.0712 - val_binary_accuracy: 0.9846

In [23]:
accuracy = model.evaluate(x_test_scaled, y_test)
print("Test Accuracy:", accuracy)

8503/8503 [==============================] - 7s 771us/step - loss: 0.0723 - binary_accuracy: 0.9845 - recall: 4.7699e-04
Test Accuracy: [0.072347491979599, 0.9845043420791626, 0.00047698544221930206]


In [24]:
# Detailed report of model to look at all metrics
from sklearn.metrics import classification_report

y_pred = model.predict(x_test_scaled)
y_pred = (y_pred >= 0.5).astype(int)
report = classification_report(y_test, y_pred)
print(report)

8503/8503 [==============================] - 6s 665us/step
              precision    recall  f1-score   support

       False       0.98      1.00      0.99    267883
        True       0.07      0.00      0.00      4193

    accuracy                           0.98    272076
   macro avg       0.53      0.50      0.50    272076
weighted avg       0.97      0.98      0.98    272076



In [25]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, confusion_matrix

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)

# Confusion Matrix
threshold = 0.5 
y_pred_binary = (y_pred >= threshold).astype(int)
cm = confusion_matrix(y_test, y_pred_binary)

# Print metrics
print("AUC:", auc)
print("Confusion Matrix:")
print(cm)

AUC: 0.5001918305637593
Confusion Matrix:
[[267858     25]
 [  4191      2]]


## Model Creation with SMOTE

In [26]:
# Applying SMOTE to address the class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=333)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [27]:
#standardize/normalize your numerical features

#use DataFrame.select_dtypes() to select float64 or int64 feature types automatically.
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns # Get a list of the column names
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

#Fit your instance ct of ColumnTransformer to the training data and at the same time transform it by using the ColumnTransformer.fit_transform()
x_train_scaled = ct.fit_transform(x_train_resampled)
x_test_scaled = ct.fit_transform(x_test) #Transform your test data instance features_test
x_val_scaled = ct.fit_transform(x_val)

In [28]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [29]:
model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall()]
)

In [30]:
history = model.fit(x_train_scaled, y_train_resampled, 
                    validation_data=(x_val_scaled, y_val),
                    epochs=10,
                    batch_size=32)

Epoch 1/10
14351/14351 [==============================] - 20s 1ms/step - loss: 0.1257 - binary_accuracy: 0.9522 - recall_1: 0.9335 - val_loss: 0.1465 - val_binary_accuracy: 0.9515 - val_recall_1: 0.1340
Epoch 2/10
14351/14351 [==============================] - 19s 1ms/step - loss: 0.0786 - binary_accuracy: 0.9733 - recall_1: 0.9599 - val_loss: 0.2082 - val_binary_accuracy: 0.9221 - val_recall_1: 0.2392
Epoch 3/10
14351/14351 [==============================] - 18s 1ms/step - loss: 0.0704 - binary_accuracy: 0.9768 - recall_1: 0.9644 - val_loss: 0.2594 - val_binary_accuracy: 0.8985 - val_recall_1: 0.2678
Epoch 4/10
14351/14351 [==============================] - 19s 1ms/step - loss: 0.0659 - binary_accuracy: 0.9786 - recall_1: 0.9671 - val_loss: 0.2676 - val_binary_accuracy: 0.8953 - val_recall_1: 0.2650
Epoch 5/10
14351/14351 [==============================] - 19s 1ms/step - loss: 0.0627 - binary_accuracy: 0.9799 - recall_1: 0.9690 - val_loss: 0.4314 - val_binary_accuracy: 0.8306 - val_re

In [31]:
accuracy = model.evaluate(x_test_scaled, y_test)
print("Test Accuracy:", accuracy)

8503/8503 [==============================] - 6s 729us/step - loss: 0.3561 - binary_accuracy: 0.8693 - recall_1: 0.3012
Test Accuracy: [0.35609349608421326, 0.8692755103111267, 0.3012163043022156]


In [32]:
y_pred = model.predict(x_test_scaled)
y_pred = (y_pred >= 0.5).astype(int)
report = classification_report(y_test, y_pred)
print(report)

8503/8503 [==============================] - 5s 627us/step
              precision    recall  f1-score   support

       False       0.99      0.88      0.93    267883
        True       0.04      0.30      0.07      4193

    accuracy                           0.87    272076
   macro avg       0.51      0.59      0.50    272076
weighted avg       0.97      0.87      0.92    272076



In [33]:

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)

# Confusion Matrix
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)
cm = confusion_matrix(y_test, y_pred_binary)

# Print metrics
print("AUC:", auc)
print("Confusion Matrix:")
print(cm)

AUC: 0.5896916369259131
Confusion Matrix:
[[235246  32637]
 [  2930   1263]]


## Model 3
Using precision as a metric and no SMOTE since it did not improve

In [34]:
#standardize/normalize your numerical features

#use DataFrame.select_dtypes() to select float64 or int64 feature types automatically.
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns # Get a list of the column names
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

#Fit your instance ct of ColumnTransformer to the training data and at the same time transform it by using the ColumnTransformer.fit_transform()
x_train_scaled = ct.fit_transform(x_train)
x_test_scaled = ct.fit_transform(x_test) #Transform your test data instance features_test
x_val_scaled = ct.fit_transform(x_val)

In [35]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [36]:
model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

In [37]:
history = model.fit(x_train_scaled, y_train, 
                    validation_data=(x_val_scaled, y_val),
                    epochs=10,
                    batch_size=32)

Epoch 1/10
7288/7288 [==============================] - 13s 2ms/step - loss: 0.0748 - binary_accuracy: 0.9846 - recall_2: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.0733 - val_binary_accuracy: 0.9846 - val_recall_2: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
7288/7288 [==============================] - 13s 2ms/step - loss: 0.0727 - binary_accuracy: 0.9846 - recall_2: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.0723 - val_binary_accuracy: 0.9846 - val_recall_2: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
7288/7288 [==============================] - 13s 2ms/step - loss: 0.0716 - binary_accuracy: 0.9846 - recall_2: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.0727 - val_binary_accuracy: 0.9846 - val_recall_2: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
7288/7288 [==============================] - 13s 2ms/step - loss: 0.0708 - binary_accuracy: 0.9846 - recall_2: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.0719 - val_binary_accuracy: 0.9846 - val_recal

In [38]:
accuracy = model.evaluate(x_test_scaled, y_test)
print("Test Accuracy:", accuracy)

8503/8503 [==============================] - 6s 751us/step - loss: 0.0720 - binary_accuracy: 0.9846 - recall_2: 0.0000e+00 - precision: 0.0000e+00
Test Accuracy: [0.07198310643434525, 0.9845815300941467, 0.0, 0.0]


In [39]:
# Detailed report of model to look at all metrics

y_pred = model.predict(x_test_scaled)
y_pred = (y_pred >= 0.5).astype(int)
report = classification_report(y_test, y_pred)
print(report)

8503/8503 [==============================] - 6s 668us/step
              precision    recall  f1-score   support

       False       0.98      1.00      0.99    267883
        True       0.00      0.00      0.00      4193

    accuracy                           0.98    272076
   macro avg       0.49      0.50      0.50    272076
weighted avg       0.97      0.98      0.98    272076



In [40]:
# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)

# Confusion Matrix
threshold = 0.5 
y_pred_binary = (y_pred >= threshold).astype(int)
cm = confusion_matrix(y_test, y_pred_binary)

# Print metrics
print("AUC:", auc)
print("Confusion Matrix:")
print(cm)

AUC: 0.499996267027023
Confusion Matrix:
[[267881      2]
 [  4193      0]]


## Model 4
Model architecture changes.  
Adding two more hidden layers.  
Adding dropout layers  
Adding regularization to hidden layers.

In [41]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.1), input_shape=(x_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.5),
    Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [42]:
model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

In [43]:
history = model.fit(x_train_scaled, y_train, 
                    validation_data=(x_val_scaled, y_val),
                    epochs=10,
                    batch_size=32)

Epoch 1/10
7288/7288 [==============================] - 18s 2ms/step - loss: 0.3259 - binary_accuracy: 0.9814 - recall_3: 0.0031 - precision_1: 0.0143 - val_loss: 0.0798 - val_binary_accuracy: 0.9846 - val_recall_3: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/10
7288/7288 [==============================] - 16s 2ms/step - loss: 0.0827 - binary_accuracy: 0.9846 - recall_3: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.0797 - val_binary_accuracy: 0.9846 - val_recall_3: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/10
7288/7288 [==============================] - 16s 2ms/step - loss: 0.0804 - binary_accuracy: 0.9846 - recall_3: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.0797 - val_binary_accuracy: 0.9846 - val_recall_3: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 4/10
7288/7288 [==============================] - 18s 2ms/step - loss: 0.0799 - binary_accuracy: 0.9846 - recall_3: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.0796 - val_binary_accuracy: 0.9846 - val

In [44]:
accuracy = model.evaluate(x_test_scaled, y_test)
print("Test Accuracy:", accuracy)

8503/8503 [==============================] - 7s 834us/step - loss: 0.0796 - binary_accuracy: 0.9846 - recall_3: 0.0000e+00 - precision_1: 0.0000e+00
Test Accuracy: [0.07960547506809235, 0.9845888614654541, 0.0, 0.0]


In [45]:
# Detailed report of model to look at all metrics

y_pred = model.predict(x_test_scaled)
y_pred = (y_pred >= 0.5).astype(int)
report = classification_report(y_test, y_pred)
print(report)

8503/8503 [==============================] - 6s 671us/step
              precision    recall  f1-score   support

       False       0.98      1.00      0.99    267883
        True       0.00      0.00      0.00      4193

    accuracy                           0.98    272076
   macro avg       0.49      0.50      0.50    272076
weighted avg       0.97      0.98      0.98    272076



c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [46]:
# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)

# Confusion Matrix
threshold = 0.5 
y_pred_binary = (y_pred >= threshold).astype(int)
cm = confusion_matrix(y_test, y_pred_binary)

# Print metrics
print("AUC:", auc)
print("Confusion Matrix:")
print(cm)

AUC: 0.5
Confusion Matrix:
[[267883      0]
 [  4193      0]]
